### Preferences model

In [1]:
import pysmile
import pysmile_license
import numpy as np
import pandas as pd

from network_functions import calculate_network_utilities, new_screening_strategy, old_screening_strategy, create_folders_logger
from simulations import plot_classification_results
from plots import plot_estimations_w_error_bars, plot_screening_counts

from preprocessing import preprocessing
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
import matplotlib.pyplot as plt

import yaml
with open('config.yaml', 'r') as file:
    cfg = yaml.safe_load(file)

import argparse
import pdb
import matplotlib
matplotlib.use('Agg')

import logging
import datetime 
import os

In [2]:
cfg['max_workers']

4

In [3]:
cfg['rel_point_cond_mut_info']

{'PE_cost': 50, 'PE_info': 4.1, 'PE_prob': 0.7}

In [4]:
# from main import main
from use_case_new_strategy import use_case_new_strategy
from full_example import full_example
from update import update_influence_diagram
from network_functions import create_folders_logger
from sens_analysis_elicitation import sens_analysis_elicitation
from sens_analysis_PE_method import sens_analysis_PE_method

In [5]:
cfg["patient_chars"]

{'Age': 'age_4_adult',
 'Sex': 'M',
 'SD': 'SD_2_normal',
 'PA': 'PA_2',
 'Smoking': 'sm_1_not_smoker',
 'BMI': 'bmi_2_normal',
 'Alcohol': 'low'}

## Use cases

### Individual cases

**Benchmark patient**

In [6]:
label = 'notebook_benchmark_patient'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = cfg["patient_chars"],
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label
)

2026-01-21 16:11:06,009 - INFO - Model type: linear
2026-01-21 16:11:06,010 - INFO - Reading network...
2026-01-21 16:11:06,026 - INFO - Calculating information values...
2026-01-21 16:12:07,976 - INFO - Eliciting value of comfort...
2026-01-21 16:12:07,977 - INFO - No elicitation of lambda values, taking default values...
2026-01-21 16:12:07,979 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2026-01-21 16:12:07,979 - INFO - Saving network...
2026-01-21 16:12:08,022 - INFO - Plotting info functions...
2026-01-21 16:12:47,538 - INFO - Calculating final utilities...
2026-01-21 16:12:47,545 - INFO - Best info: 15.299634978175163
2026-01-21 16:12:47,545 - INFO - Worst info: 4.450147717014403e-308
2026-01-21 16:12:47,547 - INFO - PE info: 4.1
2026-01-21 16:12:47,548 - INFO - v_best: 104.06788025218766
2026-01-21 16:12:47,548 - INFO - v_worst: -3.910235286299

**Benchmark patient risk seeking**

In [7]:
label = 'notebook_benchmark_risk_seeking'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = cfg["patient_chars"],
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label,
    change_risk_param=True,
    rho_param= 0.005
)

2026-01-21 16:13:20,231 - INFO - Model type: linear
2026-01-21 16:13:20,231 - INFO - Model type: linear
2026-01-21 16:13:20,235 - INFO - Reading network...
2026-01-21 16:13:20,235 - INFO - Reading network...
2026-01-21 16:13:20,255 - INFO - Calculating information values...
2026-01-21 16:13:20,255 - INFO - Calculating information values...
2026-01-21 16:14:14,361 - INFO - Eliciting value of comfort...
2026-01-21 16:14:14,361 - INFO - Eliciting value of comfort...
2026-01-21 16:14:14,363 - INFO - No elicitation of lambda values, taking default values...
2026-01-21 16:14:14,363 - INFO - No elicitation of lambda values, taking default values...
2026-01-21 16:14:14,365 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2026-01-21 16:14:14,365 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.8019

**Benchmark patient added evidence**

In [8]:
label = 'notebook_benchmark_addded_ev'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

patient_chars = cfg["patient_chars"].copy()
patient_chars["Diabetes"] = True
patient_chars["Hypertension"] = True

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = patient_chars,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label,
)

2026-01-21 16:14:50,985 - INFO - Model type: linear
2026-01-21 16:14:50,987 - INFO - Reading network...
2026-01-21 16:14:51,009 - INFO - Calculating information values...
2026-01-21 16:15:55,485 - INFO - Eliciting value of comfort...
2026-01-21 16:15:55,486 - INFO - No elicitation of lambda values, taking default values...
2026-01-21 16:15:55,488 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2026-01-21 16:15:55,489 - INFO - Saving network...
2026-01-21 16:15:55,532 - INFO - Plotting info functions...
2026-01-21 16:16:34,013 - INFO - Calculating final utilities...
2026-01-21 16:16:34,020 - INFO - Best info: 15.299634978175163
2026-01-21 16:16:34,021 - INFO - Worst info: 4.450147717014403e-308
2026-01-21 16:16:34,022 - INFO - PE info: 4.1
2026-01-21 16:16:34,023 - INFO - v_best: 104.06788025218766
2026-01-21 16:16:34,024 - INFO - v_worst: -3.910235286299

**Exogenous variable p = 0.02**

In [9]:
label = 'notebook_exogenous_var_002'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

exog_char ={"Age": "age_2_young", 
    "Sex": "M",
    "SES": "ses_0",
    "SD": "SD_1_short",
    "PA": "PA_1",
    "Depression": False,
    "Smoking": "sm_1_not_smoker",
    "Anxiety": False,
    "BMI": "bmi_1_underweight",
    "Alcohol": "high",
    "Diabetes": False,
    "Hypertension": False,
    "Hyperchol_": False,
    }


update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = exog_char,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label,
    exogenous_var_prob= 0.02
)

2026-01-21 16:16:34,304 - INFO - Model type: linear
2026-01-21 16:16:34,306 - INFO - Reading network...
2026-01-21 16:16:34,325 - INFO - Calculating information values...
2026-01-21 16:17:39,776 - INFO - Eliciting value of comfort...
2026-01-21 16:17:39,777 - INFO - No elicitation of lambda values, taking default values...
2026-01-21 16:17:39,778 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2026-01-21 16:17:39,780 - INFO - Saving network...
2026-01-21 16:17:39,840 - INFO - Plotting info functions...
2026-01-21 16:18:18,607 - INFO - Calculating final utilities...
2026-01-21 16:18:18,615 - INFO - Best info: 15.299634978175163
2026-01-21 16:18:18,616 - INFO - Worst info: 4.450147717014403e-308
2026-01-21 16:18:18,617 - INFO - PE info: 4.1
2026-01-21 16:18:18,618 - INFO - v_best: 104.06788025218766
2026-01-21 16:18:18,619 - INFO - v_worst: -3.910235286299

**Exogenous variable p = 0.1**

In [10]:
label = 'notebook_exogenous_var_01'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

exog_char ={"Age": "age_2_young", 
    "Sex": "M",
    "SES": "ses_0",
    "SD": "SD_1_short",
    "PA": "PA_1",
    "Depression": False,
    "Smoking": "sm_1_not_smoker",
    "Anxiety": False,
    "BMI": "bmi_1_underweight",
    "Alcohol": "high",
    "Diabetes": False,
    "Hypertension": False,
    "Hyperchol_": False,
    }

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = exog_char,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label,
    exogenous_var_prob= 0.1
)

2026-01-21 16:18:18,794 - INFO - Model type: linear
2026-01-21 16:18:18,796 - INFO - Reading network...
2026-01-21 16:18:18,819 - INFO - Calculating information values...
2026-01-21 16:19:22,982 - INFO - Eliciting value of comfort...
2026-01-21 16:19:22,984 - INFO - No elicitation of lambda values, taking default values...
2026-01-21 16:19:22,985 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2026-01-21 16:19:22,985 - INFO - Saving network...
2026-01-21 16:19:23,041 - INFO - Plotting info functions...
2026-01-21 16:20:01,515 - INFO - Calculating final utilities...
2026-01-21 16:20:01,523 - INFO - Best info: 15.299634978175163
2026-01-21 16:20:01,524 - INFO - Worst info: 4.450147717014403e-308
2026-01-21 16:20:01,525 - INFO - PE info: 4.1
2026-01-21 16:20:01,527 - INFO - v_best: 104.06788025218766
2026-01-21 16:20:01,527 - INFO - v_worst: -3.910235286299

### Assessing the French strategy

**Patient 1**

In [11]:
label = 'notebook_4_2_patient_1'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

patient_chars = {
    "Age": "age_5_old_adult", 
    "Sex": "M",
    "SD": "SD_2_normal",
    "PA": "PA_2",
    "Smoking": "sm_1_not_smoker",
    "BMI": "bmi_2_normal",
    "Alcohol": "low",
    "Diabetes": False,
    "Hypertension": False,
}

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = patient_chars,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label
)

2026-01-21 16:20:01,664 - INFO - Model type: linear
2026-01-21 16:20:01,666 - INFO - Reading network...
2026-01-21 16:20:01,682 - INFO - Calculating information values...
2026-01-21 16:21:08,073 - INFO - Eliciting value of comfort...
2026-01-21 16:21:08,074 - INFO - No elicitation of lambda values, taking default values...
2026-01-21 16:21:08,076 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2026-01-21 16:21:08,077 - INFO - Saving network...
2026-01-21 16:21:08,137 - INFO - Plotting info functions...
2026-01-21 16:21:46,965 - INFO - Calculating final utilities...
2026-01-21 16:21:46,973 - INFO - Best info: 15.299634978175163
2026-01-21 16:21:46,974 - INFO - Worst info: 4.450147717014403e-308
2026-01-21 16:21:46,975 - INFO - PE info: 4.1
2026-01-21 16:21:46,976 - INFO - v_best: 104.06788025218766
2026-01-21 16:21:46,977 - INFO - v_worst: -3.910235286299

**Patient 2**

In [12]:
label = 'notebook_4_2_patient_2'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

patient_chars = {
    "Age": "age_4_adult", 
    "Sex": "M",
    "SD": "SD_2_normal",
    "PA": "PA_2",
    "Smoking": "sm_1_not_smoker",
    "BMI": "bmi_2_normal",
    "Alcohol": "low",
    "Diabetes": True,
    "Hypertension": True,
}

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = patient_chars,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label
)

2026-01-21 16:21:47,216 - INFO - Model type: linear
2026-01-21 16:21:47,220 - INFO - Reading network...
2026-01-21 16:21:47,241 - INFO - Calculating information values...
2026-01-21 16:22:53,196 - INFO - Eliciting value of comfort...
2026-01-21 16:22:53,199 - INFO - No elicitation of lambda values, taking default values...
2026-01-21 16:22:53,200 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2026-01-21 16:22:53,201 - INFO - Saving network...
2026-01-21 16:22:53,257 - INFO - Plotting info functions...
2026-01-21 16:23:34,040 - INFO - Calculating final utilities...
2026-01-21 16:23:34,047 - INFO - Best info: 15.299634978175163
2026-01-21 16:23:34,049 - INFO - Worst info: 4.450147717014403e-308
2026-01-21 16:23:34,050 - INFO - PE info: 4.1
2026-01-21 16:23:34,051 - INFO - v_best: 104.06788025218766
2026-01-21 16:23:34,052 - INFO - v_worst: -3.910235286299

**Patient 3**

In [13]:
label = 'notebook_4_2_patient_3'
logger, log_dir = create_folders_logger(label = label)

patient_chars = {
    "Age": "age_4_adult", 
    "Sex": "M",
    "SD": "SD_2_normal",
    "PA": "PA_1",
    "Smoking": "sm_3_ex_smoker",
    "BMI": "bmi_3_overweight",
    "Alcohol": "high",
}

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = patient_chars,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label
)

2026-01-21 16:23:34,219 - INFO - Model type: linear
2026-01-21 16:23:34,221 - INFO - Reading network...
2026-01-21 16:23:34,242 - INFO - Calculating information values...
2026-01-21 16:24:35,600 - INFO - Eliciting value of comfort...
2026-01-21 16:24:35,603 - INFO - No elicitation of lambda values, taking default values...
2026-01-21 16:24:35,604 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2026-01-21 16:24:35,605 - INFO - Saving network...
2026-01-21 16:24:35,647 - INFO - Plotting info functions...
2026-01-21 16:25:11,627 - INFO - Calculating final utilities...
2026-01-21 16:25:11,635 - INFO - Best info: 15.299634978175163
2026-01-21 16:25:11,636 - INFO - Worst info: 4.450147717014403e-308
2026-01-21 16:25:11,637 - INFO - PE info: 4.1
2026-01-21 16:25:11,637 - INFO - v_best: 104.06788025218766
2026-01-21 16:25:11,639 - INFO - v_worst: -3.910235286299

### Designing a national personalised strategy

In [ ]:
with open('config.yaml', 'r') as file:
    cfg = yaml.safe_load(file)

label = 'notebook_personalised_screening_str'
logger, log_dir = create_folders_logger(label=label, output_dir = 'notebook/' + label, date = False, time = False)

use_case_new_strategy(
    logger = logger,
    single_run=cfg['single_run'],
    output_dir = 'notebook/' + label,
    log_dir = log_dir,
    run_label = ''
)

2026-01-21 16:25:11,808 - INFO - Configuration variables of interest:
2026-01-21 16:25:11,809 - INFO - Single run: False
2026-01-21 16:25:11,810 - INFO - Number of runs: 200
2026-01-21 16:25:11,812 - INFO - Use all variables: True
2026-01-21 16:25:11,812 - INFO - Use case with new test: False
2026-01-21 16:25:11,814 - INFO - Reading the network file...
2026-01-21 16:25:11,834 - INFO - Located at: decision_models/DM_screening_rel_point_cond_mut_info_linear.xdsl
2026-01-21 16:25:11,836 - INFO - Comfort values: 1 - 4.011942483017116, 2 - 4.168480067521723, 3 - 6.801984517973132, 4 - 7.0
2026-01-21 16:25:12,671 - INFO - All variables are kept in the dataframe for calculation of utilities.
2026-01-21 16:25:12,672 - INFO - Operational limits for the screening strategies: {'No_scr_no_col': inf, 'No_scr_col': 3000, 'gFOBT': 30000, 'FIT': 42000, 'Blood_based': 7000, 'Stool_DNA': 6000, 'CTC': 2000, 'Colon_capsule': 2000}
2026-01-21 16:25:12,715 - INFO - Multiple simulations of the tests will be 

### Benchmarking of new screening devices

In [6]:
cfg["patient_chars"]

{'Age': 'age_4_adult',
 'Sex': 'M',
 'SD': 'SD_2_normal',
 'PA': 'PA_2',
 'Smoking': 'sm_1_not_smoker',
 'BMI': 'bmi_2_normal',
 'Alcohol': 'low'}

In [7]:
label = 'notebook_benchmark_new_screening'
logger, log_dir = create_folders_logger(label=label, output_dir = 'notebook/' + label, date = False, time = False)

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = cfg["patient_chars"],
    new_test = True,
    logger = logger,
    output_dir = 'notebook/' + label
)

2026-01-27 11:02:10,072 - INFO - Model type: linear
2026-01-27 11:02:10,073 - INFO - Reading network...
2026-01-27 11:02:10,090 - INFO - Calculating information values...
2026-01-27 11:02:36,942 - INFO - Eliciting value of comfort...
2026-01-27 11:02:36,943 - INFO - No elicitation of lambda values, taking default values...
2026-01-27 11:02:36,945 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2026-01-27 11:02:36,947 - INFO - Adding values for 2 new tests...
2026-01-27 11:03:16,783 - INFO - Saving network...
2026-01-27 11:03:16,855 - INFO - Plotting info functions...
2026-01-27 11:03:22,274 - INFO - Calculating final utilities...
2026-01-27 11:03:22,284 - INFO - Best info: 15.636459410190582
2026-01-27 11:03:22,285 - INFO - Worst info: -1.4901161193847656e-08
2026-01-27 11:03:22,286 - INFO - PE info: 4.1
2026-01-27 11:03:22,286 - INFO - v_best: 106.35895

In [6]:
cfg["lambda_list"]

[4.011942483017116, 4.168480067521723, 6.801984517973132, 7.0]

In [7]:
label = 'notebook_new_test_screening_strategy'
logger, log_dir = create_folders_logger(label=label, output_dir = 'notebook/' + label, date = False, time = False)

use_case_new_strategy(
    logger = logger,
    use_case_new_test = True,
    single_run=cfg['single_run'],
    output_dir = 'notebook/' + label,
    log_dir = log_dir,
    run_label = '',
    full_analysis= False
)

2026-01-27 12:45:49,881 - INFO - Configuration variables of interest:
2026-01-27 12:45:49,888 - INFO - Single run: True
2026-01-27 12:45:49,888 - INFO - Use all variables: True
2026-01-27 12:45:49,888 - INFO - Use case with new test: True
2026-01-27 12:45:49,888 - INFO - Reading the network file...
2026-01-27 12:45:49,938 - INFO - Located at: outputs/linear_rel_point_cond_mut_info_elicitFalse_newtestTrue/decision_models/DM_screening_rel_point_cond_mut_info_linear_new_test.xdsl
2026-01-27 12:45:49,939 - INFO - Comfort values: 1 - 4.011942483017116, 2 - 7.0, 3 - 6.801984517973132, 4 - 7.0
2026-01-27 12:45:50,639 - INFO - All variables are kept in the dataframe for calculation of utilities.
2026-01-27 12:45:50,639 - WARNING - Limit for New_Test_A not found in config, setting to default (20000).
2026-01-27 12:45:50,639 - WARNING - Limit for New_Test_B not found in config, setting to default (20000).
2026-01-27 12:45:50,639 - INFO - Operational limits for the screening strategies: {'No_scr_

## Sensitivity analysis

### Sensitivity analysis for the PE

In [5]:
label = 'notebook_sens_analysis_PE_method'
logger, log_dir = create_folders_logger(label=label, output_dir = 'notebook/' + label, date = False, time = False)

sens_analysis_PE_method(
    label = label,
    output_dir = 'notebook/' + label,
    log_dir = log_dir,  
)

2025-01-27 17:50:51,100 - INFO - Reading from file: decision_models/DM_screening_rel_point_cond_mut_info_linear.xdsl
2025-01-27 17:50:51,100 - INFO - Reading from file: decision_models/DM_screening_rel_point_cond_mut_info_linear.xdsl
2025-01-27 17:50:51,102 - INFO - PE_info_array: [4.2 4.  3.8]
2025-01-27 17:50:51,102 - INFO - PE_info_array: [4.2 4.  3.8]
2025-01-27 17:50:51,103 - INFO - PE_cost_array: [ 5 10 50]
2025-01-27 17:50:51,103 - INFO - PE_cost_array: [ 5 10 50]
2025-01-27 17:50:54,889 - INFO - Params: [1.016419338142636, 0.8801528395772084, 0.03681253893587941]
2025-01-27 17:50:54,889 - INFO - Params: [1.016419338142636, 0.8801528395772084, 0.03681253893587941]
2025-01-27 17:51:59,901 - INFO - PE_info: 4.2, PE_cost: 5, DONE!
2025-01-27 17:51:59,901 - INFO - PE_info: 4.2, PE_cost: 5, DONE!
2025-01-27 17:52:00,603 - INFO - Params: [1.0260096777692866, 0.889649200484113, 0.036469730286183666]
2025-01-27 17:52:00,603 - INFO - Params: [1.0260096777692866, 0.889649200484113, 0.0364

### Sensitivity analysis for comfort parameters

In [6]:
label = 'notebook_sens_analysis_comf_param'
logger, log_dir = create_folders_logger(label = label, output_dir= 'notebook/' + label, date = False, time = False)

sens_analysis_elicitation(
    label = label,
    output_dir = 'notebook/' + label
)

2025-01-29 13:04:17,390 - INFO - Configuration variables of interest:
2025-01-29 13:04:17,390 - INFO - Configuration variables of interest:
2025-01-29 13:04:17,391 - INFO - Single run: False
2025-01-29 13:04:17,391 - INFO - Single run: False
2025-01-29 13:04:17,393 - INFO - Use all variables: True
2025-01-29 13:04:17,393 - INFO - Use all variables: True
2025-01-29 13:04:17,394 - INFO - Use case with new test: False
2025-01-29 13:04:17,394 - INFO - Use case with new test: False
2025-01-29 13:04:17,395 - INFO - PE method: {'PE_cost': 50, 'PE_info': 4.1, 'PE_prob': 0.7}
2025-01-29 13:04:17,395 - INFO - PE method: {'PE_cost': 50, 'PE_info': 4.1, 'PE_prob': 0.7}
2025-01-29 13:04:17,396 - INFO - Change lambdas? True
2025-01-29 13:04:17,396 - INFO - Change lambdas? True
2025-01-29 13:04:17,397 - INFO - Read lambda list from config? True
2025-01-29 13:04:17,397 - INFO - Read lambda list from config? True
2025-01-29 13:04:17,398 - INFO - Lambdas: [4.011942483017116, 4.168480067521723, 6.8019845

Counts: best_option
No_scr_no_col    288682
No_scr_col            0
gFOBT                 0
FIT                3201
Blood_based           0
Stool_DNA         47824
CTC                   0
Colon_capsule         0
Name: count, dtype: int64


In [7]:
label = 'notebook_sens_analysis_comf_param'
logger, log_dir = create_folders_logger(label = label, output_dir= 'notebook/' + label, date = False, time = False)

lambda_list = cfg["lambda_list"].copy()
lambda_list[2] = 6.3

sens_analysis_elicitation(
    lambda_list = lambda_list,
    label = label,
    output_dir = 'notebook/' + label
)

2025-01-29 13:07:20,934 - INFO - Configuration variables of interest:
2025-01-29 13:07:20,934 - INFO - Configuration variables of interest:
2025-01-29 13:07:20,934 - INFO - Configuration variables of interest:
2025-01-29 13:07:20,937 - INFO - Single run: False
2025-01-29 13:07:20,937 - INFO - Single run: False
2025-01-29 13:07:20,937 - INFO - Single run: False
2025-01-29 13:07:20,938 - INFO - Use all variables: True
2025-01-29 13:07:20,938 - INFO - Use all variables: True
2025-01-29 13:07:20,938 - INFO - Use all variables: True
2025-01-29 13:07:20,940 - INFO - Use case with new test: False
2025-01-29 13:07:20,940 - INFO - Use case with new test: False
2025-01-29 13:07:20,940 - INFO - Use case with new test: False
2025-01-29 13:07:20,941 - INFO - PE method: {'PE_cost': 50, 'PE_info': 4.1, 'PE_prob': 0.7}
2025-01-29 13:07:20,941 - INFO - PE method: {'PE_cost': 50, 'PE_info': 4.1, 'PE_prob': 0.7}
2025-01-29 13:07:20,941 - INFO - PE method: {'PE_cost': 50, 'PE_info': 4.1, 'PE_prob': 0.7}
2

Counts: best_option
No_scr_no_col    313960
No_scr_col            0
gFOBT                 0
FIT                   0
Blood_based           0
Stool_DNA         25747
CTC                   0
Colon_capsule         0
Name: count, dtype: int64


In [8]:
label = 'notebook_sens_analysis_comf_param'
logger, log_dir = create_folders_logger(label = label, output_dir= 'notebook/' + label, date = False, time = False)

lambda_list = cfg["lambda_list"]
lambda_list[0] = 4.8
lambda_list[1] = 5

sens_analysis_elicitation(
    lambda_list = lambda_list,
    label = label,
    output_dir = 'notebook/' + label
)

2025-01-29 13:10:02,223 - INFO - Configuration variables of interest:
2025-01-29 13:10:02,223 - INFO - Configuration variables of interest:
2025-01-29 13:10:02,226 - INFO - Single run: False
2025-01-29 13:10:02,226 - INFO - Single run: False
2025-01-29 13:10:02,228 - INFO - Use all variables: True
2025-01-29 13:10:02,228 - INFO - Use all variables: True
2025-01-29 13:10:02,230 - INFO - Use case with new test: False
2025-01-29 13:10:02,230 - INFO - Use case with new test: False
2025-01-29 13:10:02,231 - INFO - PE method: {'PE_cost': 50, 'PE_info': 4.1, 'PE_prob': 0.7}
2025-01-29 13:10:02,231 - INFO - PE method: {'PE_cost': 50, 'PE_info': 4.1, 'PE_prob': 0.7}
2025-01-29 13:10:02,233 - INFO - Change lambdas? True
2025-01-29 13:10:02,233 - INFO - Change lambdas? True
2025-01-29 13:10:02,234 - INFO - Read lambda list from config? True
2025-01-29 13:10:02,234 - INFO - Read lambda list from config? True
2025-01-29 13:10:02,235 - INFO - Lambdas: [4.8, 5, 6.801984517973132, 7.0]
2025-01-29 13:1

Counts: best_option
No_scr_no_col    179840
No_scr_col            0
gFOBT                 0
FIT                   0
Blood_based           0
Stool_DNA        159867
CTC                   0
Colon_capsule         0
Name: count, dtype: int64
